In [89]:
import pandas as pd
import ast
import json
import re
#from difflib import SequenceMatcher
#from geopy.distance import geodesic
#import numpy as np
import numpy as np
from urllib.parse import urlparse
import time

In [90]:
# Import sample data
sample_df = pd.read_csv('../input_data/processed_samples.csv')
sample_df

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': ""Goin' Postal Jacksonville""}","{'primary': 'shipping_center', 'alternate': ['...",0.996261,['http://www.goinpostaljacksonville.com/'],['https://www.facebook.com/463273470392736'],NaN,...,FL,32256-2892,7643 Gate Pkwy Ste 104,Jacksonville,FL,32256,"7643 Gate Pkwy, Jacksonville, FL, 32256-2892","7643 Gate Pkwy Ste 104, Jacksonville, FL, 32256","['shipping_center', 'freight_and_cargo_service...","['vehicle_shipping', 'courier_and_delivery_ser..."
1,1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,['http://valleytransca.com/'],['https://www.facebook.com/107150256087765'],NaN,...,CA,92021-5023,1158 N 2nd St,El Cajon,CA,92021,"1158 N 2nd St, El Cajon, CA, 92021-5023","1158 N 2nd St, El Cajon, CA, 92021","['automotive_repair', 'automotive', 'professio...","['transmission_repair', 'automotive']"
2,2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,['http://bit.ly/NelspruitMazda'],['https://www.facebook.com/113070635372645'],NaN,...,MP,1201,49 Emnotweni Avenue,Mbombela,NaN,1201,"Mazda Nelspruit, 49 Emnotweni Avenue, Riversid...","49 Emnotweni Avenue, Mbombela, 1201","['car_dealer', 'automotive', 'automotive_dealer']","['used_car_dealer', 'automotive']"
3,3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,['https://stores.redwingshoes.com/roswell-ga'],NaN,NaN,...,GA,30076,875 Mansell Rd Ste B1,Roswell,GA,30076,"875 Mansell Rd B1, Roswell, GA, 30076","875 Mansell Rd Ste B1, Roswell, GA, 30076","['shoe_store', 'online_shop', 'retail']","['shoe_store', 'online_shop', 'retail']"
4,4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,['https://centro.norauto.es/alacant/finestrat/...,['https://www.facebook.com/295967644448975'],NaN,...,NaN,03509,Calle Rei Jaume I 9,Alicante,Alacant,03509,"Carrer del Rei Jaume I, 9, Finestrat, 03509","Calle Rei Jaume I 9, Alicante, Alacant, 03509","['automotive_repair', 'car_dealer', 'automotiv...","['brake_service_and_repair', 'auto_body_shop',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,['http://www.petraccimarin.it'],['https://www.facebook.com/121776774503515'],NaN,...,36,34125,Via beccaria 3,Trieste,FVG,34125,"Viale 20 Settembre, 16, Trieste, 36, 34125","Via beccaria 3, Trieste, FVG, 34125","['lawyer', 'criminal_defense_law']",['Business and Professional Services > Office']
1996,1996,08fa80b035674b6a03379ca9c1ecea9f,50364256e4b0573431043dd4,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Farmácias Pague Menos'},"{'primary': 'pharmacy', 'alternate': ['pharmac...",0.792793,['https://www.paguemenos.com.br/'],['https://www.facebook.com/2204406756483378'],NaN,...,MS,79601-002,"Av. Antônio Trajano, 583",Três Lagoas,MS,NaN,"Avenida Antônio Trajano, 538, Três Lagoas, MS,...","Av. Antônio Trajano, 583, Três Lagoas, MS","['pharmacy', 'pharmaceutical_companies', 'beau...",['Retail >

In [91]:
# Check current types
sample_df.map(type)

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1996,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1997,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1998,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [92]:
# Assigns data types to attributes
def parse_category(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    try: return ast.literal_eval(x)
    except: return None

for c in ["sources", "names", "addresses","base_names","base_addresses","base_categories", "categories", "websites", "brand"]:
    if c in sample_df: sample_df[c] = sample_df[c].apply(parse_category)

sample_df.map(type)

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1996,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1997,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1998,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [93]:
sample_df

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],['https://www.facebook.com/463273470392736'],NaN,...,FL,32256-2892,7643 Gate Pkwy Ste 104,Jacksonville,FL,32256,"7643 Gate Pkwy, Jacksonville, FL, 32256-2892","7643 Gate Pkwy Ste 104, Jacksonville, FL, 32256","['shipping_center', 'freight_and_cargo_service...","['vehicle_shipping', 'courier_and_delivery_ser..."
1,1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],['https://www.facebook.com/107150256087765'],NaN,...,CA,92021-5023,1158 N 2nd St,El Cajon,CA,92021,"1158 N 2nd St, El Cajon, CA, 92021-5023","1158 N 2nd St, El Cajon, CA, 92021","['automotive_repair', 'automotive', 'professio...","['transmission_repair', 'automotive']"
2,2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,[http://bit.ly/NelspruitMazda],['https://www.facebook.com/113070635372645'],NaN,...,MP,1201,49 Emnotweni Avenue,Mbombela,NaN,1201,"Mazda Nelspruit, 49 Emnotweni Avenue, Riversid...","49 Emnotweni Avenue, Mbombela, 1201","['car_dealer', 'automotive', 'automotive_dealer']","['used_car_dealer', 'automotive']"
3,3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],NaN,NaN,...,GA,30076,875 Mansell Rd Ste B1,Roswell,GA,30076,"875 Mansell Rd B1, Roswell, GA, 30076","875 Mansell Rd Ste B1, Roswell, GA, 30076","['shoe_store', 'online_shop', 'retail']","['shoe_store', 'online_shop', 'retail']"
4,4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[https://centro.norauto.es/alacant/finestrat/c...,['https://www.facebook.com/295967644448975'],NaN,...,NaN,03509,Calle Rei Jaume I 9,Alicante,Alacant,03509,"Carrer del Rei Jaume I, 9, Finestrat, 03509","Calle Rei Jaume I 9, Alicante, Alacant, 03509","['automotive_repair', 'car_dealer', 'automotiv...","['brake_service_and_repair', 'auto_body_shop',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,[http://www.petraccimarin.it],['https://www.facebook.com/121776774503515'],NaN,...,36,34125,Via beccaria 3,Trieste,FVG,34125,"Viale 20 Settembre, 16, Trieste, 36, 34125","Via beccaria 3, Trieste, FVG, 34125","['lawyer', 'criminal_defense_law']",['Business and Professional Services > Office']
1996,1996,08fa80b035674b6a03379ca9c1ecea9f,50364256e4b0573431043dd4,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Farmácias Pague Menos'},"{'primary': 'pharmacy', 'alternate': ['pharmac...",0.792793,[https://www.paguemenos.com.br/],['https://www.facebook.com/2204406756483378'],NaN,...,MS,79601-002,"Av. Antônio Trajano, 583",Três Lagoas,MS,NaN,"Avenida Antônio Trajano, 538, Três Lagoas, MS,...","Av. Antônio Trajano, 583, Três Lagoas, MS","['pharmacy', 'pharmaceutical_companies', 'beau...",['Retail > Pharmacy']
1997

# Attribute Conflation Labeling Strategy

The Project B dataset contains pairs of places where each row represents the same physical place from two different data sources. The "base_" prefixed columns represent Place A (source A) and the non-prefixed columns represent Place B (source B). 

Goal: Create a labeling system to determine which attributes are more reliable for building a universal places database.

## Strategy Overview

Labeling Approach:
**Per-Attribute Binary Labels**: For each attribute type, determine if Place A or Place B has the better/more reliable value

- Extra: An aggregate score indicating which place (A or B) has better overall data quality

## Labeled Attributes:
- Names
- Addresses 
- Categories
- Websites
- Socials
- Emails
- Phones
- Brand

In [ ]:
class AttributeQualityEvaluator:
    """
    Evaluates quality of attributes for place conflation labeling
    """
    
    def __init__(self):
        self.attribute_weights = {
            'names': 0.25,
            'addresses': 0.20, 
            'categories': 0.15,
            'websites': 0.10,
            'phones': 0.10,
            'emails': 0.10,
            'socials': 0.05,
            'brand': 0.05
        }
        
        # Address standardization preferences
        self.direction_abbrevs = {'north': 'n', 'south': 's', 'east': 'e', 'west': 'w',
                                   'northeast': 'ne', 'northwest': 'nw', 'southeast': 'se', 'southwest': 'sw'}
        self.street_type_abbrevs = {
            'street': 'st', 'avenue': 'ave', 'boulevard': 'blvd', 'drive': 'dr',
            'road': 'rd', 'lane': 'ln', 'court': 'ct', 'circle': 'cir',
            'parkway': 'pkwy', 'highway': 'hwy', 'place': 'pl', 'terrace': 'ter',
            'trail': 'trl', 'way': 'way', 'plaza': 'plz', 'pike': 'pike'
        }
        
        # City name abbreviations to penalize
        self.city_abbrevs = {
            'saint': 'st', 'fort': 'ft', 'mount': 'mt', 'port': 'pt'
        }
        
        # Business suffixes to penalize
        self.business_suffixes = [
            'llc', 'l.l.c.', 'inc', 'inc.', 'incorporated',
            'co', 'co.', 'company', 'corp', 'corp.', 'corporation',
            'ltd', 'ltd.', 'limited', 'plc', 'p.l.c.'
        ]
    
    def check_name_quality(self, name_text):
        """Check name quality - reward simplicity, proper capitalization; penalize trailing numbers and business suffixes"""
        if not name_text:
            return 0.0
        
        name_str = str(name_text).strip()
        bonus = 0.0
        
        # Penalize names with trailing numbers (e.g., "The UPS Store 6285")
        # Look for space followed by 3+ digits at the end
        if re.search(r'\s+\d{3,}$', name_str):
            bonus -= 0.15
        
        # Penalize business suffixes (LLC, Inc, Co, etc.)
        name_lower = name_str.lower()
        for suffix in self.business_suffixes:
            # Check for suffix at end of name (with optional comma/period)
            if re.search(r',?\s+' + re.escape(suffix) + r'\.?$', name_lower):
                bonus -= 0.10
                break
        
        # Reward shorter/simpler names (fewer words)
        # Split on whitespace to count words
        words = name_str.split()
        num_words = len(words)
        
        if num_words <= 3:
            bonus += 0.10  # Short, simple names (1-3 words)
        elif num_words <= 5:
            bonus += 0.05  # Moderate length (4-5 words)
        # No bonus for 6+ words (verbose names)
        
        # Reward proper Title Case (first letter of each word capitalized)
        # Check if all words start with uppercase (excluding articles/prepositions that may be lowercase)
        if len(words) > 0:
            # Check if name follows Title Case pattern
            # Allow common lowercase words like "and", "of", "the", "a", "an", "at", "in", "on"
            lowercase_exceptions = {'and', 'of', 'the', 'a', 'an', 'at', 'in', 'on', 'or', 'but', 'for', 'to'}
            
            properly_capitalized = True
            for i, word in enumerate(words):
                # First word should always be capitalized
                if i == 0:
                    if not word[0].isupper():
                        properly_capitalized = False
                        break
                else:
                    # Other words should be capitalized unless they're common lowercase words
                    if word.lower() not in lowercase_exceptions and not word[0].isupper():
                        properly_capitalized = False
                        break
            
            if properly_capitalized:
                bonus += 0.10
        
        return bonus
    
    def evaluate_name_quality(self, name_data):
        """Evaluate name quality based on completeness and structure"""
        if not name_data or pd.isna(name_data):
            return 0.0
            
        if isinstance(name_data, dict):
            score = 0.0
            primary_name = name_data.get('primary')
            
            # Primary name exists
            if primary_name:
                score += 0.6
                # Bonus for reasonable length
                if 5 <= len(primary_name) <= 100:
                    score += 0.2
                
                # Check name quality (penalize trailing numbers and business suffixes, reward simplicity and capitalization)
                name_quality = self.check_name_quality(primary_name)
                score += name_quality
            
            # Alternative names exist
            if name_data.get('alternatives') and len(name_data['alternatives']) > 0:
                score += 0.2
            
            return max(min(score, 1.0), 0.0)
        
        # If it's a string, basic scoring
        if isinstance(name_data, str):
            base_score = 0.8 if 5 <= len(name_data) <= 100 else 0.4
            
            # Apply name quality check
            name_quality = self.check_name_quality(name_data)
            score = base_score + name_quality
            
            return max(min(score, 1.0), 0.0)
        
        return 0.0
    
    def check_address_standardization(self, street_text):
        """Check if address uses standard abbreviations (returns bonus score)"""
        if not street_text:
            return 0.0
        
        street_lower = str(street_text).lower()
        bonus = 0.0
        
        # Check for abbreviated directions (prefer "W" over "West")
        for full, abbrev in self.direction_abbrevs.items():
            # Check if abbreviated form is used (surrounded by word boundaries)
            if re.search(r'\b' + re.escape(abbrev) + r'\b', street_lower):
                bonus += 0.05
                break
            # Penalize if full form is used when abbreviation exists
            elif re.search(r'\b' + re.escape(full) + r'\b', street_lower):
                bonus -= 0.05
                break
        
        # Check for abbreviated street types (prefer "Dr" over "Drive")
        for full, abbrev in self.street_type_abbrevs.items():
            # Reward abbreviated form
            if re.search(r'\b' + re.escape(abbrev) + r'\b', street_lower):
                bonus += 0.05
                break
            # Penalize full form
            elif re.search(r'\b' + re.escape(full) + r'\b', street_lower):
                bonus -= 0.05
                break
        
        return bonus
    
    def check_city_quality(self, city_text):
        """Check city name quality - penalize abbreviations (prefer 'Saint Louis' over 'St. Louis')"""
        if not city_text:
            return 0.0
        
        city_lower = str(city_text).lower()
        penalty = 0.0
        
        # Check for abbreviated city name prefixes (penalize abbreviations)
        for full, abbrev in self.city_abbrevs.items():
            # Penalize if abbreviated form is used (e.g., "St." or "St")
            if re.search(r'\b' + re.escape(abbrev) + r'\.?\b', city_lower):
                penalty -= 0.05
                break
            # Reward if full form is used (e.g., "Saint")
            elif re.search(r'\b' + re.escape(full) + r'\b', city_lower):
                penalty += 0.05
                break
        
        return penalty
    
    def is_complete_street_address(self, street_text):
        """Check if street address has both a number and street name (complete address)"""
        if not street_text:
            return False
        
        street_str = str(street_text).strip()
        
        # Check for street number at the beginning (handles ranges like 123-125)
        has_number = bool(re.match(r'^\d+', street_str))
        
        # Check for street name (at least one letter after potential number)
        # This ensures we have actual content beyond just the number
        has_street_name = bool(re.search(r'\d+\s+[A-Za-z]', street_str))
        
        return has_number and has_street_name
    
    def evaluate_address_quality(self, address_data):
        """Evaluate address quality based on completeness and structure"""
        if not address_data or pd.isna(address_data):
            return 0.0
            
        if isinstance(address_data, list) and len(address_data) > 0:
            addr = address_data[0]
            score = 0.0
            
            # Street address component (CRITICAL - heavily penalize if incomplete)
            if addr.get('freeform'):
                street_text = addr.get('freeform')
                
                # Check if address is complete (has number + street name)
                if self.is_complete_street_address(street_text):
                    # Complete address: full score + standardization bonus
                    score += 0.5
                    standardization_bonus = self.check_address_standardization(street_text)
                    score += standardization_bonus
                else:
                    # Incomplete address (e.g., "USF Library" or just building name)
                    # Heavily penalize - only give minimal credit for having something
                    score += 0.05
            else:
                # No street address at all - this is very bad
                score += 0.0
            
            # City component (prefer full names over abbreviations)
            city_value = addr.get('locality') or addr.get('city')
            if city_value:
                score += 0.15
                # Add quality bonus/penalty for city name format
                city_quality = self.check_city_quality(city_value)
                score += city_quality
            
            # Postal code component
            if addr.get('postcode'):
                pc = str(addr.get('postcode'))
                # Prefer simple 5-digit ZIP over extended ZIP+4 format (Yelp doesn't use ZIP+4)
                if re.match(r'^\d{5}$', pc):  # Simple 5-digit ZIP (preferred, matches Yelp)
                    score += 0.15
                elif re.match(r'^\d{5}-\d{4}$', pc):  # ZIP+4 format (penalize - not used in Yelp)
                    score -= 0.10
                else:
                    score += 0.10
            
            # Country component
            if addr.get('country'):
                score += 0.05
            
            # State/region component
            if addr.get('region') or addr.get('state'):
                score += 0.10
            
            return min(score, 1.0)
        
        return 0.0
    
    def evaluate_categories_quality(self, categories_data):
        """Evaluate categories quality based on number and specificity"""
        if not categories_data or pd.isna(categories_data):
            return 0.0
        
        if isinstance(categories_data, list):
            if len(categories_data) == 0:
                return 0.0
            
            score = 0.0
            # Base score for having categories
            score += 0.4
            
            # Bonus for multiple categories (up to a point)
            num_cats = min(len(categories_data), 5)
            score += (num_cats - 1) * 0.1
            
            # Bonus for specific categories (longer names often more specific)
            avg_length = np.mean([len(str(cat)) for cat in categories_data])
            if avg_length > 10:
                score += 0.2
            
            return min(score, 1.0)
        
        return 0.0
    
    def evaluate_website_quality(self, website_data):
        """Evaluate website quality based on validity and completeness - reward HTTPS"""
        if not website_data or pd.isna(website_data):
            return 0.0
        
        if isinstance(website_data, list):
            valid_urls = 0
            has_https = False
            
            for url in website_data:
                if self.is_valid_url(url):
                    valid_urls += 1
                    # Check for HTTPS
                    if str(url).lower().startswith('https://'):
                        has_https = True
            
            if valid_urls > 0:
                # Base score for having valid URL(s)
                score = 0.5 + (valid_urls - 1) * 0.2
                
                # Bonus for HTTPS
                if has_https:
                    score += 0.15
                
                return min(score, 1.0)
        
        return 0.0
    
    def evaluate_phone_quality(self, phone_data):
        """Evaluate phone number quality - reward country codes"""
        if not phone_data or pd.isna(phone_data):
            return 0.0
        
        if isinstance(phone_data, list):
            valid_phones = 0
            has_country_code = False
            
            for phone in phone_data:
                if self.is_valid_phone(phone):
                    valid_phones += 1
                    # Check for country code (starts with +)
                    if str(phone).strip().startswith('+'):
                        has_country_code = True
            
            if valid_phones > 0:
                # Base score for having valid phone(s)
                score = 0.5 + (valid_phones - 1) * 0.2
                
                # Bonus for country code
                if has_country_code:
                    score += 0.4
                
                return min(score, 1.0)
        
        return 0.0
    
    def evaluate_email_quality(self, email_data):
        """Evaluate email quality"""
        if not email_data or pd.isna(email_data):
            return 0.0
        
        if isinstance(email_data, list):
            valid_emails = 0
            for email in email_data:
                if self.is_valid_email(email):
                    valid_emails += 1
            
            if valid_emails > 0:
                return min(0.6 + (valid_emails - 1) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_social_quality(self, social_data):
        """Evaluate social media quality"""
        if not social_data or pd.isna(social_data):
            return 0.0
        
        if isinstance(social_data, list) and len(social_data) > 0:
            return min(0.3 + len(social_data) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_brand_quality(self, brand_data):
        """Evaluate brand quality"""
        if not brand_data or pd.isna(brand_data):
            return 0.0
        
        if isinstance(brand_data, str) and len(brand_data) > 2:
            return 0.8
        
        return 0.0
    
    def is_valid_url(self, url):
        """Check if URL is valid"""
        try:
            result = urlparse(str(url))
            return all([result.scheme, result.netloc])
        except:
            return False
    
    def is_valid_phone(self, phone):
        """Check if phone number looks valid"""
        if not phone:
            return False
        
        # Remove common formatting
        clean_phone = re.sub(r'[^\d+]', '', str(phone))
        # Basic validation - reasonable length
        return 7 <= len(clean_phone) <= 15
    
    def is_valid_email(self, email):
        """Check if email looks valid"""
        if not email:
            return False
        
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(email_pattern, str(email)))

# Initialize evaluator
evaluator = AttributeQualityEvaluator()


In [95]:
def generate_attribute_labels(row):
    """
    Generate per-attribute and overall quality labels for a place pair
    
    Returns:
    - Per-attribute labels: 1 if Place A (base_) is better, 0 if Place B is better
    - Overall label: 1 if Place A is better overall, 0 if Place B is better overall
    - Quality scores for transparency (both individual and overall)
    - Golden dataset row with winning attributes
    """
    
    labels = {}
    place_a_scores = {}
    place_b_scores = {}
    golden_row = {}
    
    # Always include sample_idx in golden dataset
    golden_row['sample_idx'] = row['sample_idx']
    
    # Evaluate each attribute pair
    attribute_pairs = [
        ('names', 'base_names'),
        ('addresses', 'base_addresses'), 
        ('categories', 'base_categories'),
        ('websites', 'base_websites'),
        ('phones', 'base_phones'),
        ('emails', 'base_emails'),
        ('socials', 'base_socials'),
        ('brand', 'base_brand')
    ]
    
    for attr_b, attr_a in attribute_pairs:
        if attr_a in row.index and attr_b in row.index:
            # Get quality scores
            if attr_b == 'names':
                score_a = evaluator.evaluate_name_quality(row[attr_a])
                score_b = evaluator.evaluate_name_quality(row[attr_b])
            elif attr_b == 'addresses':
                score_a = evaluator.evaluate_address_quality(row[attr_a])
                score_b = evaluator.evaluate_address_quality(row[attr_b])
            elif attr_b == 'categories':
                score_a = evaluator.evaluate_categories_quality(row[attr_a])
                score_b = evaluator.evaluate_categories_quality(row[attr_b])
            elif attr_b == 'websites':
                score_a = evaluator.evaluate_website_quality(row[attr_a])
                score_b = evaluator.evaluate_website_quality(row[attr_b])
            elif attr_b == 'phones':
                score_a = evaluator.evaluate_phone_quality(row[attr_a])
                score_b = evaluator.evaluate_phone_quality(row[attr_b])
            elif attr_b == 'emails':
                score_a = evaluator.evaluate_email_quality(row[attr_a])
                score_b = evaluator.evaluate_email_quality(row[attr_b])
            elif attr_b == 'socials':
                score_a = evaluator.evaluate_social_quality(row[attr_a])
                score_b = evaluator.evaluate_social_quality(row[attr_b])
            elif attr_b == 'brand':
                score_a = evaluator.evaluate_brand_quality(row[attr_a])
                score_b = evaluator.evaluate_brand_quality(row[attr_b])
            else:
                score_a = score_b = 0.0
            
            # Store scores
            place_a_scores[attr_b] = score_a
            place_b_scores[attr_b] = score_b
            
            # Add individual quality scores to output
            labels[f'quality_score_{attr_b}_base'] = score_a
            labels[f'quality_score_{attr_b}'] = score_b
            labels[f'quality_diff_{attr_b}'] = score_a - score_b
            
            # Create binary label (1 if A is better, 0 if B is better)
            # If scores are very close (within 0.1), prefer the one with actual data
            if abs(score_a - score_b) < 0.1:
                # Tie-breaking: prefer non-null data
                has_data_a = row[attr_a] is not None and not pd.isna(row[attr_a])
                has_data_b = row[attr_b] is not None and not pd.isna(row[attr_b])
                
                if has_data_a and not has_data_b:
                    labels[f'label_{attr_b}'] = 1
                elif has_data_b and not has_data_a:
                    labels[f'label_{attr_b}'] = 0
                else:
                    # True tie
                    labels[f'label_{attr_b}'] = 2
            else:
                labels[f'label_{attr_b}'] = 1 if score_a > score_b else 0
            
            # Build golden dataset row - select winning attribute
            # Label 0 or 2 (tie) -> use non-base; Label 1 -> use base
            if labels[f'label_{attr_b}'] == 1:
                golden_row[attr_b] = row[attr_a]
            else:  # 0 or 2
                golden_row[attr_b] = row[attr_b]
    
    # Calculate overall weighted score
    total_weight_a = 0
    total_weight_b = 0
    
    for attr_b in place_a_scores.keys():
        if attr_b in evaluator.attribute_weights:
            weight = evaluator.attribute_weights[attr_b]
            total_weight_a += weight * place_a_scores[attr_b]
            total_weight_b += weight * place_b_scores[attr_b]
    
    # Overall label
    labels['label_overall'] = 1 if total_weight_a > total_weight_b else 0
    
    # Add overall quality scores for transparency
    labels['quality_score_a'] = total_weight_a
    labels['quality_score_b'] = total_weight_b
    labels['quality_difference'] = total_weight_a - total_weight_b
    
    # Add golden row data to labels for collection
    for key, value in golden_row.items():
        labels[f'golden_{key}'] = value
    
    return pd.Series(labels)

# Test the labeling function on first few rows
print("Testing labeling function on first 3 rows:")
test_labels = sample_df.head(3).apply(generate_attribute_labels, axis=1)
print(test_labels)


Testing labeling function on first 3 rows:
   quality_score_names_base  quality_score_names  quality_diff_names  \
0                       0.8                  0.8                 0.0   
1                       0.8                  0.8                 0.0   
2                       0.8                  0.8                 0.0   

   label_names  quality_score_addresses_base  quality_score_addresses  \
0            2                          1.00                     0.75   
1            2                          1.00                     0.80   
2            2                          0.75                     0.45   

   quality_diff_addresses  label_addresses  quality_score_categories_base  \
0                    0.25                1                            0.0   
1                    0.20                1                            0.0   
2                    0.30                1                            0.0   

   quality_score_categories  ...  quality_difference  golden_sampl

In [96]:
# Apply labeling to entire dataset
print("Generating labels and golden dataset for all 2000 place pairs...")
start_time = time.time()

# Generate labels for all rows
labels_df = sample_df.apply(generate_attribute_labels, axis=1)

# Combine with original data
labeled_sample_df = pd.concat([sample_df, labels_df], axis=1)

# Extract golden dataset columns
golden_columns = ['golden_sample_idx', 'golden_names', 'golden_addresses', 'golden_categories', 
                  'golden_websites', 'golden_phones', 'golden_emails', 'golden_socials', 'golden_brand']
golden_dataset = labels_df[golden_columns].copy()

# Rename columns to remove 'golden_' prefix
golden_dataset.columns = [col.replace('golden_', '') for col in golden_dataset.columns]

end_time = time.time()
print(f"Labeling time: {end_time - start_time:.2f} seconds")

# Display labeling summary
print("\n=== LABELING SUMMARY ===")
print(f"Total place pairs labeled: {len(labeled_sample_df)}")

print("\nPer-attribute label distribution:")
attribute_labels = [col for col in labels_df.columns if col.startswith('label_') and col != 'label_overall']
for label_col in attribute_labels:
    attr_name = label_col.replace('label_', '')
    count_a = (labels_df[label_col] == 1).sum()
    count_b = (labels_df[label_col] == 0).sum()
    count_tie = (labels_df[label_col] == 2).sum()
    print(f"  {attr_name:12s}: Place A (base_) better: {count_a:4d} | Place B better: {count_b:4d} | Tie: {count_tie:4d}")

print(f"\nOverall quality distribution:")
overall_a = (labels_df['label_overall'] == 1).sum()
overall_b = (labels_df['label_overall'] == 0).sum()
print(f"  Place A (base_) better overall: {overall_a}")
print(f"  Place B better overall: {overall_b}")

print(f"\nQuality score statistics:")
print(f"  Place A mean quality: {labels_df['quality_score_a'].mean():.3f}")
print(f"  Place B mean quality: {labels_df['quality_score_b'].mean():.3f}")
print(f"  Mean quality difference (A-B): {labels_df['quality_difference'].mean():.3f}")
print(f"  Quality difference std: {labels_df['quality_difference'].std():.3f}")

print(f"\n=== GOLDEN DATASET SUMMARY ===")
print(f"Total records: {len(golden_dataset)}")
print(f"Columns: {list(golden_dataset.columns)}")

# Show distribution of data completeness
print("\nData completeness per attribute:")
for col in golden_dataset.columns:
    if col != 'sample_idx':
        non_null_count = golden_dataset[col].notna().sum()
        completeness_pct = (non_null_count / len(golden_dataset)) * 100
        print(f"  {col:12s}: {non_null_count:4d} / {len(golden_dataset)} ({completeness_pct:.1f}%)")


Generating labels and golden dataset for all 2000 place pairs...
Labeling time: 0.52 seconds

=== LABELING SUMMARY ===
Total place pairs labeled: 2000

Per-attribute label distribution:
  names       : Place A (base_) better:   20 | Place B better:   35 | Tie: 1945
  addresses   : Place A (base_) better:  841 | Place B better:  192 | Tie:  967
  categories  : Place A (base_) better:    4 | Place B better:    0 | Tie: 1996
  websites    : Place A (base_) better:   34 | Place B better: 1668 | Tie:  298
  phones      : Place A (base_) better:  106 | Place B better:    1 | Tie: 1893
  emails      : Place A (base_) better:  219 | Place B better:    0 | Tie: 1781
  socials     : Place A (base_) better:    5 | Place B better: 1358 | Tie:  637
  brand       : Place A (base_) better: 1954 | Place B better:    5 | Tie:   41

Overall quality distribution:
  Place A (base_) better overall: 1001
  Place B better overall: 999

Quality score statistics:
  Place A mean quality: 0.380
  Place B mean qu

In [97]:
# Create detailed analysis and examples

# 1. Show examples of clear winners
print("=== EXAMPLES OF CLEAR QUALITY DIFFERENCES ===")

# Find rows with largest quality differences
top_a_indices = labels_df.nlargest(3, 'quality_difference').index
top_b_indices = labels_df.nsmallest(3, 'quality_difference').index

print("\nTop 3 cases where Place A (base_) is clearly better:")
for i, idx in enumerate(top_a_indices):
    row = labeled_sample_df.iloc[idx]
    print(f"\nExample {i+1} (Row {idx}):")
    print(f"  Quality difference: {row['quality_difference']:.3f}")
    print(f"  Place A name: {row['base_names']}")
    print(f"  Place B name: {row['names']}")
    print(f"  Place A address: {row['base_addresses']}")
    print(f"  Place B address: {row['addresses']}")

print("\n\nTop 3 cases where Place B is clearly better:")
for i, idx in enumerate(top_b_indices):
    row = labeled_sample_df.iloc[idx]
    print(f"\nExample {i+1} (Row {idx}):")
    print(f"  Quality difference: {row['quality_difference']:.3f}")
    print(f"  Place A name: {row['base_names']}")
    print(f"  Place B name: {row['names']}")
    print(f"  Place A websites: {row['base_websites']}")
    print(f"  Place B websites: {row['websites']}")

# 2. Analyze attribute correlation
print("\n\n=== ATTRIBUTE CORRELATION ANALYSIS ===")

# Create correlation matrix for binary labels
label_cols = [col for col in labels_df.columns if col.startswith('label_') and col != 'label_overall']
correlation_matrix = labels_df[label_cols].corr()

print("Correlation between attribute labels:")
print(correlation_matrix.round(3))

# 3. Quality score distribution
print(f"\n=== QUALITY SCORE DISTRIBUTION ===")
print(f"Quality difference distribution:")
print(f"  Min: {labels_df['quality_difference'].min():.3f}")
print(f"  25th percentile: {labels_df['quality_difference'].quantile(0.25):.3f}")
print(f"  Median: {labels_df['quality_difference'].median():.3f}")
print(f"  75th percentile: {labels_df['quality_difference'].quantile(0.75):.3f}")
print(f"  Max: {labels_df['quality_difference'].max():.3f}")

# 4. Identify challenging cases (very close quality scores)
close_cases = labeled_sample_df[abs(labeled_sample_df['quality_difference']) < 0.01]
print(f"\nChallenging cases (quality difference < 0.01): {len(close_cases)} out of {len(labeled_sample_df)}")

=== EXAMPLES OF CLEAR QUALITY DIFFERENCES ===

Top 3 cases where Place A (base_) is clearly better:

Example 1 (Row 1124):
  Quality difference: 0.160
  Place A name: {'primary': "Ongkeco's Hobby Shop", 'common': {}, 'rules': []}
  Place B name: {'primary': "Ongkeco'S Hobby Shop"}
  Place A address: [{'freeform': '2488 Taft Ave.', 'locality': 'Manila', 'region': 'Philippines', 'country': 'PH', 'postcode': '1004'}]
  Place B address: [{'freeform': 'Estrada St 2486', 'locality': 'Manila', 'postcode': '1004', 'country': 'PH'}]

Example 2 (Row 1788):
  Quality difference: 0.150
  Place A name: {'primary': 'Clinique de la Chaussure', 'common': {}, 'rules': []}
  Place B name: {'primary': 'Clinique de la Chaussure'}
  Place A address: [{'freeform': '6 Bis rue Saint-Georges', 'locality': 'Mussidan', 'region': 'Dordogne', 'country': 'FR', 'postcode': '24400'}]
  Place B address: [{'freeform': '6B Rue Saint-Georges', 'locality': 'Mussidan', 'postcode': '24400', 'country': 'FR'}]

Example 3 (Row

In [ ]:
def export_labeled_dataset(df, filename='../output_data/algorithm_labels.csv'):
    """Export the labeled dataset with comprehensive metadata"""
    
    # Select columns to export (include sample_idx from the original CSV)
    # Include base and non_base attributes for reference and future comparison
    base_columns = ['sample_idx', 'id', 'base_id']
    
    # Add all extracted/processed and original attributes for comparison to Yelp dataset
    attribute_columns = [
        # Names - extracted and original
        'name_primary', 'base_name_primary',
        'names', 'base_names',
        # Categories - extracted and original
        'categories_list', 'base_categories_list',
        'categories', 'base_categories',
        # Addresses - extracted and original
        'address_string', 'base_address_string',
        'addresses', 'base_addresses',
        # Individual address components
        'address_street', 'address_city', 'address_state', 'address_postcode',
        'base_address_street', 'base_address_city', 'base_address_state', 'base_address_postcode',
        # Websites
        'websites', 'base_websites',
        # Contact info
        'phones', 'base_phones',
        'emails', 'base_emails',
        # Social media
        'socials', 'base_socials',
        # Brand
        'brand', 'base_brand'
    ]
    
    # Add label and quality columns
    label_quality_columns = [col for col in df.columns if col.startswith('label_') or col.startswith('quality_')]
    
    # Combine all columns
    export_columns = base_columns + attribute_columns + label_quality_columns
    
    # Filter to only include columns that exist in the dataframe
    export_columns = [col for col in export_columns if col in df.columns]
    
    simplified_df = df[export_columns].copy()
    
    simplified_df.to_csv(filename, index=False)
    print(f"✓ Exported {len(simplified_df)} labeled records to {filename}")
    print(f"  Columns exported: {len(export_columns)}")
    print(f"  Includes: identifiers, all base and non_base attributes (extracted & original), labels, and quality scores")
    

export = export_labeled_dataset(labeled_sample_df)


✓ Exported 2000 labeled records to ../output_data/weighted_algorithm_labels.csv
  Columns exported: 69
  Includes: identifiers, all base and non_base attributes (extracted & original), labels, and quality scores


In [99]:
# Export golden dataset to CSV
golden_dataset.to_csv('../output_data/golden_dataset.csv', index=False)
print(f"✓ Exported golden dataset to ../output_data/golden_dataset.csv")
print(f"  Records exported: {len(golden_dataset)}")
print(f"  Columns: {len(golden_dataset.columns)}")
print(f"\nFirst 5 rows of golden dataset:")
print(golden_dataset.head())


✓ Exported golden dataset to ../output_data/golden_dataset.csv
  Records exported: 2000
  Columns: 9

First 5 rows of golden dataset:
   sample_idx                                     names  \
0           0  {'primary': 'Goin' Postal Jacksonville'}   
1           1        {'primary': 'Valley Transmission'}   
2           2            {'primary': 'Mazda Nelspruit'}   
3           3     {'primary': 'Red Wing - Roswell, GA'}   
4           4             {'primary': 'Norauto España'}   

                                           addresses  \
0  [{'freeform': '7643 Gate Pkwy Ste 104', 'local...   
1  [{'freeform': '1158 N 2nd St', 'locality': 'El...   
2  [{'freeform': '49 Emnotweni Avenue', 'locality...   
3  [{'freeform': '875 Mansell Rd B1', 'locality':...   
4  [{'freeform': 'Calle Rei Jaume I 9', 'locality...   

                                          categories  \
0  {'primary': 'shipping_center', 'alternate': ['...   
1  {'primary': 'automotive_repair', 'alternate': ...   
2  {'p

In [100]:
# Calculate non-null percentages for each attribute in golden dataset
print("=== GOLDEN DATASET ATTRIBUTE COMPLETENESS ===\n")

# Get all columns except sample_idx
attribute_cols = [col for col in golden_dataset.columns if col != 'sample_idx']

# Calculate non-null counts and percentages
completeness_data = []
for col in attribute_cols:
    non_null_count = golden_dataset[col].notna().sum()
    total_count = len(golden_dataset)
    percentage = (non_null_count / total_count) * 100
    completeness_data.append({
        'Attribute': col,
        'Non-Null Count': non_null_count,
        'Total Count': total_count,
        'Percentage': percentage
    })

# Create DataFrame and sort by percentage (descending)
completeness_df = pd.DataFrame(completeness_data)
completeness_df = completeness_df.sort_values('Percentage', ascending=False)

# Display the results
print(completeness_df.to_string(index=False))

# Summary statistics
print(f"\n=== SUMMARY ===")
print(f"Average completeness: {completeness_df['Percentage'].mean():.2f}%")
print(f"Median completeness: {completeness_df['Percentage'].median():.2f}%")
print(f"Most complete attribute: {completeness_df.iloc[0]['Attribute']} ({completeness_df.iloc[0]['Percentage']:.2f}%)")
print(f"Least complete attribute: {completeness_df.iloc[-1]['Attribute']} ({completeness_df.iloc[-1]['Percentage']:.2f}%)")


=== GOLDEN DATASET ATTRIBUTE COMPLETENESS ===

 Attribute  Non-Null Count  Total Count  Percentage
     names            2000         2000      100.00
 addresses            2000         2000      100.00
    phones            1997         2000       99.85
categories            1988         2000       99.40
     brand            1959         2000       97.95
  websites            1746         2000       87.30
   socials            1397         2000       69.85
    emails             219         2000       10.95

=== SUMMARY ===
Average completeness: 83.16%
Median completeness: 98.68%
Most complete attribute: names (100.00%)
Least complete attribute: emails (10.95%)
